### The data files

# Producing the pre v post data

In [ ]:
from importlib.machinery import SourceFileLoader 
IM = SourceFileLoader("InterconditionalMethods", "/home/rullman/scRNAmuscleProject/algorithms/scRNAalgoes/src/scMethods.py").load_module() 

In [ ]:
prefix = "/home/rullman/scRNAmuscleProject/bigproj/private/dataoutput/FINAL_FIGURES"
data = sc.read_h5ad("091120_adata_newmerge_rank.h5ad")

***Celltype based clusters***

In [ ]:
celltypes = {
    "Endothelial Cells": ["Endo", "Bad"],
    "Lymphocyte Cells": ["T-cells 13", "T-cells 15"],
    "Pericyte": ["Pericyte 5", "Pericyte 18"],
    "Mesenchymal Cells": ["Mesenchymal merged", "Mesenchymal cell 11", "CD16"],
    "Myoblast Cells": ["Myoblast 12", "Myoblast 3", "Satellite cell"],
    "Monocyte Cells": ["CD14+"],
}

***Adding the celltype based clusters as a separate column***

In [ ]:
projID = "090521"

In [ ]:
projID = "100521" 

In [ ]:
data.obs["celltype"] = np.nan

Iterate through the key-value pair subsetting for the values equating them to the key

In [ ]:
for key in celltypes:
    for el in celltypes[key]:
        data.obs.celltype[data.obs.annotations.str.contains(el)] = key

In [ ]:
data.obs.celltype = data.obs.celltype.astype("category")

### Checking if the data is normalized

In [ ]:
IM.test_random_log1p_countsnormalize(data)

In [ ]:
data_prepost = IM.diffxpyInterconditionalIterative(
        data, 
        clusterColumn='celltype', 
        projID=projID, 
        use_raw=False, 
        method="wilcoxon"
    )

The batch keys that will be used for the diffexp analysis

In [ ]:
batchKeys = ['Endothelial Cells_batch', 'Lymphocyte Cells_batch', 'Mesenchymal Cells_batch', 'Monocyte Cells_batch', 'Myoblast Cells_batch', 'Pericyte_batch']

In [ ]:
IM.add_pct_manual(data, clusters = data.obs.celltype.cat.categories,  
clusterColumn="celltype", keys=batchKeys, out=f"{projID}_prevpost_rankwilcox.xlsx", save='excel') 

> Produced 090511_prevpost_rankwilcox.xlsx

In [ ]:
# data.write("090521_adata_celltype_w_prevpost.h5ad", compression = "gzip")
data.write("100521_adata_celltype_w_prevpost.h5ad", compression = "gzip")

# Plotting the pre v post boxplots anew

In [ ]:
import scanpy as sc

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from importlib.machinery import SourceFileLoader 
# IM = SourceFileLoader("InterconditionalMethods", "/home/rullman/scRNAmuscleProject/algorithms/scRNAalgoes/src/scMethods.py").load_module() 
IM = SourceFileLoader("InterconditionalMethods", "/home/alirassolie/Documents/prwork/klinfys/scmuscle/scRNAalgoes/src/scMethods.py").load_module() 

##### Prefixes

In [ ]:
prefix = r"C:\Users\Admin\Documents\prwork\klinfys\manuscript_v2\manuscript\datasets\paper\figures\figure_1"

In [ ]:
prefix = "~/Documents/prwork/bioinfo/data/manuscript/datasets/paper/figures/figure_1"
prefix = "~/Documents/manuscript/datasets/paper/figures/figure_1"

##### Importing and processing the data anew

In [ ]:
localPath = "/home/alirassolie/Documents/arbete/mint_mobile"

In [ ]:
data = sc.read_h5ad(f"{localPath}/120521_adata_celltype_prevpost_and_markers")

In [ ]:
data.X = data.X.toarray()

***Extracting the values for the boxplots***

In [ ]:
filter_adata = lambda adata, col, f_str: adata[adata.obs[col].str.contains(f_str)]

In [ ]:
def gene_distrib(d, genes, s_str, cluster_column):
    """Returns dataframe with data molten for each gene and a given
    column
    
    Args:
        d: AnnData scanpy object
        
        genes: list object of strings corresponding to gene symbols
        
        s_str: string to filter d
    
    """
    tm = d[d.obs[cluster_column].str.contains(s_str)]
    g_vals = pd.DataFrame(tm[:, genes].X)
    print(g_vals)
    g_vals.columns = genes
    g_vals['celltag'] = tm.obs.index.values
    g_vals['batch'] = tm.obs.batch.values
    g_vals[cluster_column] = s_str
    g_vals_melt = pd.melt(g_vals, ['batch', cluster_column, "celltag"])
    return g_vals_melt

In [ ]:
# Testing
# Removed the ones that looked bad in the plot, we need to check them in the files..
testGenes = ["TIMP3", "ACTB", "UBC", "CALM1"]
# Normalized
testDf = gene_distrib(data, testGenes, s_str='Endothelial Cells', cluster_column='celltype')

In [ ]:
testDf

Boxplot function

In [ ]:
def diffxp_boxplot(axs, i:int, d:"anndata", cluster_column, genes):
    """Returns dataframe with data molten for each gene and a given
    column
    
    Args:
        axs: list with matplotlib axes objec
        
        i: int index
        
        d: AnnData scanpy object
        
        genes: list object of strings corresponding to gene symbols
        
        cluster_column
    """
    s_str = order[i]
    g_vals_melt = gene_distrib(d, genes, s_str, cluster_column)
    sns.boxplot(
        data=g_vals_melt,
        x='variable',
        y='value',
        hue='batch',
        ax=axs[i],
        palette=sns.color_palette(['#ca7c80', '#fde3d0']),
        showfliers=False
    )
    molten_celltypes.append(g_vals_melt)
    axs[i].set_title('')
    axs[i].set_ylabel(s_str, rotation=0, labelpad=100, fontsize=20)
    axs[i].set_xlabel('')
    axs[i].get_legend().remove()
    # sns.stripplot(data=g_vals_melt, x='variable', y='value', hue='batch', ax=axs)
    # axs.set_ylim(0, 6)
    return axs

In [ ]:
# 100521 boxplots
order = ["Endothelial Cells", "Lymphocyte Cells", "Pericyte", "Mesenchymal Cells", "Myoblast Cells", "Monocyte Cells", "remove"]
genes_t_inc_r = ["ACTB", "CALM1", "CDH5", "ENG", "FOS", "HSP90AA1", "JUNB", "TIMP3", "UBC", "IFITM2", "IRF1", "LMNA", "MT2A", "MYF5", "NCL", "NNMT", "SOCS3", "SQSTM1"]

sns.set_style('ticks')
fig, axs = plt.subplots(nrows=2, figsize = (20,5))
axs = axs.flatten()
molten_celltypes = []

for i in range(len(order)):
    axs = diffxp_boxplot(axs, i, data, cluster_column="celltype", genes=genes_t_inc_r)

sns.despine(trim=False)
# plt.savefig('./FIGURE_2/130521_split_boxplot.png')
plt.show()

#### On the raw

In [ ]:
raw = data.raw.to_adata()

In [ ]:
raw.X = raw.X.toarray()

In [ ]:
# 100521 boxplots
data.obs.celltype = data.obs.celltype.str.replace(' 0 ', '')

order = ["Endothelial Cells", "Lymphocyte Cells", "Pericyte", "Mesenchymal Cells", "Myoblast Cells", "Monocyte Cells", "remove"]
genes_t_inc_r = ["ACTB", "CALM1", "CDH5", "ENG", "FOS", "HSP90AA1", "JUNB", "TIMP3", "UBC", "IFITM2", "IRF1", "LMNA", "MT2A", "MYF5", "NCL", "NNMT", "SOCS3", "SQSTM1"]

sns.set_style('ticks')
fig, axs = plt.subplots(nrows=int(len(col_ord)/2), figsize = (20,5))
axs = axs.flatten()
molten_celltypes = []

for i in range(len(order)):
    axs = diffxp_boxplot(axs, i, raw, cluster_column="celltype", genes=genes_t_inc_r)

sns.despine(trim=False)
# plt.savefig('./FIGURE_2/100521_split_boxplot_raw.png')
plt.show()

#### Violin plots instead

In [ ]:
from matplotlib.pyplot import rc_context
import matplotlib as mpl

In [ ]:
sc.settings.figdir = "FINAL_REVIEW_FIGURE"

#### palette

In [ ]:
celltypePalette = {
    "Endo": "#1f77b4",
    "Peri": "#2ca02c",
    "Myo": "#9467bd",
    "Mono": "#8c564b",
    "T": "#ff7f0e",
    "Mes": "#d62728"
}

In [ ]:
def plotHatchedViolin(
        data, save=None, figsize=(7,5), 
        inner=None, cut=0, v=None, **kwargs
):
    with rc_context({'figure.figsize': figsize}):
        sns.violinplot(data=data, 
                           hue = "batch", 
                           x="variable", y="value", 
                           split=False, 
                           rotation=45, 
                           #cut=cut,
                           linewidth=2.2,
                           edgecolor="black",
                           #inner=inner,
                           showmeans=True,
                           ax=v,
                           **kwargs
                          )
        # sns.boxplot(data=data, hue="batch", x="variable", y="value", width=0.2, color="white",boxprops={'zorder': 2},ax=v)
        # sns.stripplot(data=test_df, hue = "batch", x="variable", y="value", split=True, size=1)
        for i, violin in enumerate(v.findobj(mpl.collections.PolyCollection)):
            if i % 2:
                violin.set_hatch("////")

        v.legend_.findobj(mpl.patches.Rectangle)[1].set_hatch("///")
        v.set_xticklabels(v.get_xticklabels(), rotation=30)
        v.set_ylim(ymin=0)

In [ ]:
def plotBoxplot(
        data, save=None, figsize=(7,5), 
        inner=None, cut=0, v=None, **kwargs
):
    with rc_context({'figure.figsize': figsize}):
        sns.boxplot(data=data, 
                    hue = "batch", 
                    x="variable", y="value", 
                   linewidth=2.2,
                   showmeans=True,
                   ax=v,
                    
                    **kwargs
                          )
        #sns.stripplot(data=test_df, hue = "batch", x="variable", y="value", split=True, size=1)
        v.set_xticklabels(v.get_xticklabels(), rotation=30)
        v.set_ylim(ymin=0)

In [ ]:
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

##### Endothelial

In [ ]:
# Removed the ones that looked bad in the plot, we need to check them in the files..
genes_end = ["TIMP3", "ACTB", "UBC", "CALM1"]
# Normalized
endo_df = gene_distrib(data, genes_end, 'Endothelial Cells', 'celltype')

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(10,6))
plotHatchedViolin(data=endo_df, figsize=(8,6), 
                  #save = "/home/alirassolie/Documents/misc/endo_prevpost.pdf",
                  #save="ax",
                  palette = [celltypePalette["Endo"]],
                  v=axs[0]
                 )

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(10,6))
plotBoxplot(data=endo_df, figsize=(8,6), 
                  #save = "/home/alirassolie/Documents/misc/endo_prevpost.pdf",
                  #save="ax",
                  #palette = [celltypePalette["Endo"]],
                  v=axs[0]
                 )

In [ ]:
genes = ["MYL2", "ACTA1", "CCDC85B", "TCAP"]

In [ ]:
# Normalized
test_df = gene_distrib(data, genes, 'Endothelial Cells', 'celltype')

In [ ]:
plotHatchedViolin(data=test_df, figsize=(8,6), 
                  save = "/home/alirassolie/Documents/misc/endo_prevpost_down.pdf",
                  palette = [celltypePalette["Endo"]],
                  ax=
                 )

##### Myoblasts

In [ ]:
genes_myo = ["RPL10","RACK1", "FTL", "MT1X"]

# ALl of the myoblasts merged
myo_df = gene_distrib(data, genes_myo, 'Myoblast Cells', 'celltype')

In [ ]:
fig, axs=plt.subplots(ncols=2, figsize=(20,5))
plotHatchedViolin(data=myo_df, 
                  figsize=(8,6),
                  palette=[celltypePalette["Myo"]],
                  v=axs[0]
                 )

In [ ]:
# Only satellite cells
sat_df = gene_distrib(data, genes_myo, 'Satellite cell', 'annotations')

In [ ]:
fig, axs=plt.subplots(ncols=2, figsize=(20,5))
plotHatchedViolin(data=myo_df, 
                  figsize=(8,6),
                  palette=[celltypePalette["Myo"]],
                  v=axs[0]
                 )

##### Lymphocyte cells

In [ ]:
genes_lymph = ["FTH1", "IFITM2", "CALM1", "HLA-E"]

# ALl of the myoblasts merged
lymph_df = gene_distrib(data, genes_lymph, 'Lymphocyte Cells', 'celltype')

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(10,6))

plotHatchedViolin(data=lymph_df, 
                  #save = "/home/alirassolie/Documents/misc/lymph_prevpost.pdf",
                  save="ax",
                  figsize = (8,6),
                  palette=[celltypePalette["T"]],
                  v=axs[0]
                 )

In [ ]:
# ALl of the myoblasts merged
test_df = gene_distrib(
                data, 
                ["B2M", "EEF1A1", "HLA-A", "ACTA1"], 
                'Lymphocyte Cells', 
                'celltype')

In [ ]:
plotHatchedViolin(data=test_df, 
                  save = "/home/alirassolie/Documents/misc/lymph_prevpost_down.pdf",
                  figsize = (8,6),
                  palette=[celltypePalette["T"]]
                 )

##### Mesenchymal cells

In [ ]:
genes_mes = ["VIM", "UBC",  "GPX4", "ACVRL1"]

# All of the myoblasts merged
mes_df = gene_distrib(data, genes_mes, 'Mesenchymal Cells', 'celltype')

In [ ]:
plotHatchedViolin(data= mes_df,
                  #save = "/home/alirassolie/Documents/misc/mesench_prevpost.pdf",
                  save="ax",
                  figsize=(8,6),
                  palette=[celltypePalette["Mes"]]
                 )

In [ ]:
# ALl of the myoblasts merged
test_df = gene_distrib(
                data, 
                ["JUND", "TCF4","CKM", "IGFBP5"], 
                'Mesenchymal Cells', 
                'celltype')

In [ ]:
plotHatchedViolin(data=test_df,
                  save = "/home/alirassolie/Documents/misc/mesench_prevpost_down.pdf",
                  figsize=(8,6),
                  palette=[celltypePalette["Mes"]]
                 )

##### Pericytes

In [ ]:
genes_peri = ["ACTA2", "ACTB", "MYL6", "TPM1"]

# All of the myoblasts merged
peri_df = gene_distrib(data, genes_peri, 'Pericyte', 'celltype')

In [ ]:
plotHatchedViolin(data = test_df,
                  #save = "/home/alirassolie/Documents/misc/peric_prevpost.pdf",
                  save="ax",
                  palette=[celltypePalette["Peri"]],
                  bw=0.3
                 )

##### Monocyte cells

In [ ]:
genes_mono = ["NFKBIA", "CTSS", "HLA-C", "SLC25A6"]

# ALl of the myoblasts merged
mono_df = gene_distrib(data, genes_mono, 'Monocyte Cells', 'celltype')

In [ ]:
plotHatchedViolin(data = mono_df, 
                  #save = "/home/alirassolie/Documents/misc/Monocytes_prevpost.pdf",
                  save="ax",
                  figsize=(7,5),
                  palette=[celltypePalette["Mono"]],
                  bw=0.1,
                  #inner="point",
                 )

In [ ]:
celltypePalette["Mono"]

In [ ]:
vio_dict = {
    "Mono": plotHatchedViolin(data = mono_df, 
                  #save = "/home/alirassolie/Documents/misc/Monocytes_prevpost.pdf",
                  save="ax",
                  figsize=(7,5),
                  palette=[celltypePalette["Mono"]]
                 ),
    "Peri": plotHatchedViolin(data = test_df,
                  #save = "/home/alirassolie/Documents/misc/peric_prevpost.pdf",
                  save="ax",
                  palette=[celltypePalette["Peri"]]
                 ),
    "Mesen": plotHatchedViolin(data= mes_df,
                  #save = "/home/alirassolie/Documents/misc/mesench_prevpost.pdf",
                  save="ax",
                  figsize=(8,6),
                  palette=[celltypePalette["Mes"]]
                 ),
    "Lymph": plotHatchedViolin(data=lymph_df, 
                  #save = "/home/alirassolie/Documents/misc/lymph_prevpost.pdf",
                  save="ax",
                  figsize = (8,6),
                  palette=[celltypePalette["T"]]
                 ),
    "Endo": plotHatchedViolin(data=endo_df, figsize=(8,6), 
                  # save = "/home/alirassolie/Documents/misc/endo_prevpost.pdf",
                    save="ax",
                  palette = [celltypePalette["Endo"]]
                 )
}

In [ ]:
vio_kwargs = [
    dict(data = mono_df, 
    figsize=(7,5),
    palette=[celltypePalette["Mono"]]
    ),
    dict(data = peri_df,
    figsize=(8,6),
    palette=[celltypePalette["Peri"]]
    ),
    dict(data= mes_df,
    figsize=(8,6),
    palette=[celltypePalette["Mes"]]
    ),
    dict(data=lymph_df, 
    figsize = (8,6),
    palette=[celltypePalette["T"]]
    ),
    dict(data=endo_df, 
    figsize=(8,6), 
    palette = [celltypePalette["Endo"]]
    ),
    dict(data=myo_df, 
    figsize=(8,6),
    palette=[celltypePalette["Myo"]],
    #v=axs[0]
    )
]

In [ ]:
fig, axs = plt.subplots(ncols=3, nrows=2, figsize=(15,8))
axs = axs.flatten()
for i, ax in enumerate(axs):
    try: plotHatchedViolin(v=ax, **vio_kwargs[i])
    except Exception as e: print(e)
        
# plt.savefig("/home/alirassolie/Documents/misc/panel_prevpost.pdf")

In [ ]:
fig, axs = plt.subplots(ncols=3, nrows=2, figsize=(20,8))
axs = axs.flatten()
for i, ax in enumerate(axs):
    try: plotHatchedViolin(v=ax, **vio_kwargs[i])
    except Exception as e: print(e)
        
plt.savefig("/home/alirassolie/Documents/arbete/mint_mobile/finalreview/Figure_3_boxplot_inner_290922.pdf")

In [ ]:
writer = pd.ExcelWriter("/home/alirassolie/Documents/arbete/mint_mobile/finalreview/Figure_3_boxplot_inner_290922.xlsx", engine="xlsxwriter")
endo_df.to_excel(writer, sheet_name="Endothelial")
myo_df.to_excel(writer, sheet_name="Myoblast")
lymph_df.to_excel(writer, sheet_name="Lymph")
mes_df.to_excel(writer, sheet_name="Mesenchymal")
peri_df.to_excel(writer, sheet_name="Pericyte")
mono_df.to_excel(writer, sheet_name="Monocyte")
writer.save()

In [ ]:
writer = pd.ExcelWriter("/home/alirassolie/Documents/arbete/mint_mobile/finalreview/Supplement_10_Figure_3_boxplot_inner_290922.xlsx", engine="xlsxwriter")
pd.concat([endo_df, myo_df, lymph_df, mes_df, peri_df, mono_df]).to_excel(writer, sheet_name="Figure_3_violin")
writer.save()

In [ ]:
pre_colors = [
    ""
]

# UMAP with celltype

In [ ]:
prefix = "~/Documents/manuscript/datasets/paper/figures/figure_1"

In [ ]:
data = pd.read_csv(f"{prefix}/020720_adata_metadata_dpt.csv")

In [ ]:
for i in data.annotations.unique(): print(i)

In [ ]:
data["celltype"] = np.nan

In [ ]:
celltypes = {
    "Endothelial Cells": ["Endo", "Bad"],
    "Lymphocyte Cells": ["T-cells 13", "T-cells 15"],
    "Pericyte": ["Pericyte 5", "Pericyte 18"],
    "Mesenchymal Cells": ["Mesenchymal merged", "Mesenchymal cell 11", "CD16"],
    "Myoblast Cells": ["Myoblast 12", "Myoblast 3", "Satellite cell"],
    "Monocyte Cells": ["CD14+"],
    "remove": ["Myoblast 19", "Remove this cluster"],
}

In [ ]:
for key in celltypes:
    for el in celltypes[key]:
        data.celltype[data.annotations.str.contains(el)] = key

In [ ]:
plt.rcParams['figure.figsize'] = (15,15)
fig, ax = plt.subplots(ncols=1)

# sns.scatterplot(data.x_um, data.y_um)
sns.scatterplot(data.x_um, data.y_um, s=24, color='black', edgecolors=None, linewidth=0, ax=ax)

#sns.scatterplot(data.x_um, data.y_um, s=7, color='lightgrey', linewidth=0, ax=ax)
t = sns.scatterplot(data.x_um, data.y_um, s=5, linewidth=0, hue=data['celltype'], ax=ax, palette='tab10')
sns.despine(left=True, bottom=True, ax=ax)

#### Removing the cells we do not want

In [ ]:
fig1_outpref = "/home/ali/Documents/prwork/bioinfo/data/manuscript/datasets/paper/figures/figure_1/FIGURE_1_EPS"
fig1_outpref = "/home/ali/Documents/prwork/bioinfo/data/manuscript/datasets/paper/figures/figure_1/FIGURE_1_EPS"

In [ ]:
filtered_data = data[~data.celltype.str.contains("(?i)remove")]

In [ ]:
plt.rcParams['figure.figsize'] = (15,15)
fig, ax = plt.subplots(ncols=1)

# sns.scatterplot(filtered_data.x_um, filtered_data.y_um)
sns.scatterplot(filtered_data.x_um, filtered_data.y_um, s=24, color='black', edgecolors=None, linewidth=0, ax=ax)

#sns.scatterplot(filtered_data.x_um, filtered_data.y_um, s=7, color='lightgrey', linewidth=0, ax=ax)
t = sns.scatterplot(filtered_data.x_um, filtered_data.y_um, s=5, linewidth=0, hue=filtered_data['celltype'], ax=ax, palette='tab10')
sns.despine(left=True, bottom=True, ax=ax)

plt.savefig(f"{fig1_outpref}/080521_filtered_deep_umap.eps")

In [ ]:
filtered_data.to_csv(f"{fig1_outpref}/080521_filtered_dataset.csv")

# Ontologies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import OrderedDict
from scipy.cluster.hierarchy import dendrogram, linkage

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# File from 240720
# goExEffect = pd.read_csv("/home/alirassolie/Documents/misc/goTerms_EF_custom_bck.csv")
goExEffect = pd.read_csv("/home/alirassolie/Documents/prwork/klinfys/data/go_terms_exercise_effect.csv")

In [ ]:
goExEffect

In [ ]:
onto = goExEffect.pivot(index = "GO.clusterP", columns="Cell.type")["GO.qval"].fillna(0.0)
onto

In [ ]:
ontDict = {
    "Pericyte_5":[
        " GO_WOUND_HEALING",
        " GO_CONNECTIVE_TISSUE_DEVELOPMENT"
        # " GO_BIOLOGICAL_ADHESION", #
    ],
    "Endothelial_2":[
        " GO_CELL_CYCLE_G2_M_PHASE_TRANSITION",
        " GO_ENERGY_RESERVE_METABOLIC_PROCESS",
    ],
    "T-cells_13":[
        #" GO_CELLULAR_CATABOLIC_PROCESS",#
        " GO_NUCLEAR_TRANSCRIBED_MRNA_CATABOLIC_PROCESS_NONSENSE_MEDIATED_DECAY" #
    ],
    "Mesenchymal_merged":[
        " GO_REGENERATION",
        " GO_IMMUNE_RESPONSE_REGULATING_CELL_SURFACE_RECEPTOR_SIGNALING_PATHWAY",
    ],
    "Endothelial_Cell_0":[
        " GO_TISSUE_REGENERATION",
        " GO_CYTOSKELETON_ORGANIZATION" # 
    ],
    "Myoblast_12":[
        " GO_ACTIN_MYOSIN_FILAMENT_SLIDING",
        " GO_MUSCLE_SYSTEM_PROCESS"
    ],
    "Myoblast_3":[
        " GO_MUSCLE_SYSTEM_PROCESS",
        " GO_SKELETAL_MUSCLE_CONTRACTION"
    ],
    "Satellite_cell":[
        #" GO_REGENERATION", # 
        " GO_NEGATIVE_REGULATION_OF_CELL_DEATH",   
    ],
    "CD14+_Monocyte":[
        " GO_CELLULAR_AMIDE_METABOLIC_PROCESS", # 
        " GO_ORGANONITROGEN_COMPOUND_BIOSYNTHETIC_PROCESS" #
    ]

}

In [ ]:
ontList = [i for k in ontDict.values() for i in k]

In [ ]:
ontListFiltered = set(ontList) & set(onto.index)

In [ ]:
sns.heatmap(onto[onto.index.isin(ontList)].T, cmap="hot", square=True)

In [ ]:
ontoFilteredDF = onto[onto.index.isin(ontListFiltered)]
ontoFilteredDF
ontoFilteredDF.to_csv("/home/alirassolie/Documents/prwork/klinfys/data/review_fig_3_dotplot_qvals.csv")

In [ ]:
ontoFilteredDF = ontoFilteredDF.loc[:, reversed(["Pericyte 5", "Endothelial 2", "Endothelial Cell 0", "Mesenchymal merged", "Satellite cell", "Myoblast 12", "Myoblast 3", "T-cells 15", "CD14+ Monocyte"])]

In [ ]:
subclusterPalette = {
    "CD14+ Monocyte": "#8c564b", 
    "T-cells 15": "#ff7f0e", 
    "Myoblast 3": "#9467bd", 
    "Myoblast 12": "#9467bd", 
    "Satellite cell": "#9467bd", 
    "Mesenchymal merged": "#d62728", 
    # "Mesenchymal_cell_11": "#d62728", 
    "Pericyte 5": "#2ca02c", 
    "Endothelial Cell 0": "#1f77b4", 
    "Endothelial 2": "#1f77b4"
}

In [ ]:
def dotplot(df, scale=10, color=None):
    #gs_kw = dict(width_ratios=[4, 1], height_ratios=[4,1])
    plt.rcParams["figure.dpi"] = 90
    if color == None: color="lightblue"
    fig, ax = plt.subplots(ncols=1, nrows=1) #, gridspec_kw=gs_kw)
    # axs = axs.flatten()
    # ax = axs[0]

    for c in df.columns:
        tmp = -np.log(df.loc[:, c])*scale
        tmp[tmp == np.inf] = 0
        
        # print(df)
        plt.scatter(
            y=np.full(len(df.index), c),
            x=df.index,
            s=(tmp == 0).astype(int)*1,
            marker="1",
            color="black"
        )
        plt.scatter(
            y=np.full(len(df.index), c),
            x=df.index,
            s=np.log(tmp)*5,
            color="black",
            #fontweight=1
        )
        plt.scatter(
            y=np.full(len(df.index), c),
            x=df.index,
            s=tmp,
            color=color[c]
        )

    plt.xticks(rotation=90)
    # plt.savefig("/home/alirassolie/Documents/misc/dotplot.pdf")

In [ ]:
# By ontology, index
linked = linkage(ontoFilteredDF, optimal_ordering=True)
a = dendrogram(linked)
ind_y = [int(i) for i in a['ivl']]

plt.show()

In [ ]:
dotplot(ontoFilteredDF.iloc[[i for i in reversed(ind_y)]], scale=3, color=subclusterPalette)

# Ligand Receptor analysis

In [ ]:
import pandas as pd

In [ ]:
LRdf = pd.read_csv("/home/alirassolie/Documents/misc/human_lr_pair.txt", delimiter="\t")

In [ ]:
LRdf

In [ ]:
diffxp = pd.ExcelFile("/home/alirassolie/Documents/misc/100521_prevpost_rankwilcox.xlsx")

In [ ]:
help(diffxp.parse)

In [ ]:
endo_df = diffxp.parse("Endothelial Cells", header=0, skiprows=1, index_col="Unnamed: 0")
endo_df = endo_df[endo_df.scores.notnull()]

In [ ]:
endo_df = endo_df[(endo_df.scores > 0) & (endo_df.pvals_adj <= 0.05)]

In [ ]:
set(endo_df.index) & set(LRdf.ligand_gene_symbol)

In [ ]:
list(LRdf[LRdf.ligand_gene_symbol == "COL4A1"].receptor_gene_symbol)

In [ ]:
def identify_receptors(excel):
    sheetnames = excel.sheet_names
    receptors = {}
    for name in sheetnames:
        df = diffxp.parse(name, header=0, skiprows=1, index_col="Unnamed: 0")
        df = df[df.scores.notnull()]
        df = df[(df.scores > 0) & (df.pvals_adj <= 0.05)]
        receptors[name] = list(set(df.index) & set(LRdf.receptor_gene_symbol))
    
    return receptors
    
    

In [ ]:
receptorDict = identify_receptors(diffxp)

In [ ]:
receptorDict

In [ ]:
def identify_ligands(excel):
    sheetnames = excel.sheet_names
    ligandsDict = {}
    for name in sheetnames:
        df = diffxp.parse(name, header=0, skiprows=1, index_col="Unnamed: 0")
        df = df[df.scores.notnull()]
        df = df[(df.scores > 0) & (df.pvals_adj <= 0.05)]
        ligandsDict[name] = list(set(df.index) & set(LRdf.ligand_gene_symbol))
    
    return ligandsDict
        

In [ ]:
ligandsDict = identify_ligands(diffxp)

In [ ]:
ligandsDict

In [ ]:
def identify_LR_pair(ligands, receptors, LRdf):
    out = {}
    for ligand_key in ligands.keys():
        out[ligand_key] = list()
        for i in range(len(ligands[ligand_key])):
            r = LRdf[LRdf.ligand_gene_symbol == ligands[ligand_key][i]].receptor_gene_symbol.values
            for rec in r:
                for receptor_key in receptors.keys():
                    if rec in receptors[receptor_key]: 
                        out[ligand_key].append(set((ligands[ligand_key][i], (receptor_key, rec))))
    
    return out
            
            
            
        

In [ ]:
LRdf[LRdf.ligand_gene_symbol == ligandsDict["Endothelial Cells"][0]].receptor_gene_symbol.values

In [ ]:
o = set()

In [ ]:
identify_LR_pair(ligandsDict, receptorDict, LRdf)

In [ ]:
LRdf[LRdf.lr_pair.str.contains('LGALS1')]

# Background 

In [ ]:
data.obs.columns

In [ ]:
counts = {}
for i, cl in enumerate(data.obs.celltype.cat.categories):
    print(cl)
    tmp = data[data.obs.celltype == cl]
    counts[cl] = np.count_nonzero(tmp.raw[:, tmp.var_names].X.toarray(), axis=0)
    if i>0: 
        assert np.array_equal(tmp.var_names,genes)
        print('True')
    genes = tmp.var_names

bg = pd.DataFrame(columns = genes)
for k in counts.keys():
    bg.loc[k, :] = counts[k]

bg = bg.T

In [ ]:
bg

In [ ]:
data.obs